In [2]:
# -- coding: utf-8 -

In [20]:
!ls

drive  sample_data


In [1]:
import os
import logging
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

import dialogflow

In [11]:
updater = Updater("1428756614:AAGDiCk5A6VR1vzQ_zHY67maif_8XONS898", use_context=True)  # Токен API к Telegram
dispatcher = updater.dispatcher
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/content/drive/MyDrive/Colab Notebooks/janetry-ogqx-7b1c8abd3f15.json'# скачнный JSON


DIALOGFLOW_PROJECT_ID = 'janetry-ogqx' #PROJECT ID из DialogFlow 
DIALOGFLOW_LANGUAGE_CODE = 'ru' # язык
SESSION_ID = 'janeTry'  # ID бота из телеграма

In [12]:
def startCommand1(update: Update, context: CallbackContext):
    update.message.reply_text('Добрый день!')

    
def textMessage1(update: Update, context: CallbackContext):
    session_client = dialogflow.SessionsClient()
    session = session_client.session_path(DIALOGFLOW_PROJECT_ID, SESSION_ID)
    text_input = dialogflow.types.TextInput(text=update.message.text , language_code=DIALOGFLOW_LANGUAGE_CODE)
    query_input = dialogflow.types.QueryInput(text=text_input)
    
    try:
        response = session_client.detect_intent(session=session, query_input=query_input)
    except InvalidArgument:
         raise

    text = response.query_result.fulfillment_text
    if text:
        update.message.reply_text(response.query_result.fulfillment_text)
    else:
        update.message.reply_text('Что?')

In [13]:
# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler("start", startCommand1))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, textMessage1))

# Start the Bot
updater.start_polling()
updater.idle()

In [ ]:
# Хендлеры
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
# Добавляем хендлеры в диспетчер
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
# Начинаем поиск обновлений
updater.start_polling(clean=True)
# Останавливаем бота, если были нажаты Ctrl + C
updater.idle()

In [2]:
import os
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd

In [3]:


#Small preprocess of the answers

question = None
written = False

with open("prepared_answers.txt", "w") as fout:
    with open("/content/drive/MyDrive/Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


In [4]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [26]:


# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("/content/drive/MyDrive/Otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


In [5]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [27]:
sentences = [i for i in sentences if len(i) > 2]
modelFT = FastText(sentences=sentences, size=100, min_count=1, window=5)
modelFT.save("ft_model")

In [6]:
modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(100 ,'angular')

In [28]:
modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with open("prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelFT:
                vector_ft += modelFT[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
            
        counter += 1

ft_index.build(10)
ft_index.save('speaker.ann')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


True

In [8]:
ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('speaker.ann') 

True

In [9]:
ft_index.get_nns_by_vector(np.zeros(100), 2)

[20216, 30337]

In [10]:
shop_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ProductsDataset.csv")


shop_data['text'] = shop_data['title'] + " " + shop_data["descrirption"]
shop_data['text'] = shop_data['text'].apply(lambda x: preprocess_txt(str(x)))
shop_data.head()

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, orby, новый, носить, реал, кра..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, 4042, брюки, новый, знать, мер..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, 42, размерсостояние, отли..."


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(ngram_range=(1, 2))

In [12]:
import pickle

with open('index_speaker.pkl', 'rb') as f:
     index_map = pickle.load(f)

In [13]:
idxs = set(np.random.randint(0, len(index_map), len(shop_data)))
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in idxs]
positive_texts = [" ".join(val) for val in shop_data['text'].values]

In [14]:
dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, stratify=labels,
                                                    random_state=13)

In [16]:
x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression().fit(x_train_vec, y_train)

In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.979596174282678

In [39]:

import pickle

with open("index_speaker.pkl", "wb") as f:
    pickle.dump(index_map, f)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer().fit(X_train)

In [25]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}
idfs

{'платье': 9.23323750070527,
 'размер': 6.937341395862021,
 '444648': 10.842675413139371,
 'новый': 11.248140521247535,
 'детский': 11.248140521247535,
 'розовомголубомзелёный': 11.248140521247535,
 'красный': 11.248140521247535,
 'цветен': 11.248140521247535,
 'рост': 10.55499334068759,
 '8090100110': 11.248140521247535,
 '120': 11.248140521247535,
 'pлепто': 11.248140521247535,
 'пус': 11.248140521247535,
 'кита': 11.248140521247535,
 'йский': 11.248140521247535,
 'лата': 11.248140521247535,
 'leptopus': 11.248140521247535,
 'chinensis': 11.248140521247535,
 'вид': 11.248140521247535,
 'цветковый': 10.842675413139371,
 'растение': 11.248140521247535,
 'род': 11.248140521247535,
 'лептопус': 11.248140521247535,
 'семейство': 11.248140521247535,
 'филлантовыеp': 11.248140521247535,
 'h2ботанический': 11.248140521247535,
 'описаниеh2': 11.248140521247535,
 'pкустарник': 11.248140521247535,
 'высота': 11.248140521247535,
 '80': 11.248140521247535,
 'смотреть': 11.248140521247535,
 'густо

In [42]:
ft_index_shop = annoy.AnnoyIndex(100 ,'angular')

midf = np.mean(tfidf_vect.idf_)

index_map_shop = {}
counter = 0

for i in tqdm_notebook(range(len(shop_data))):
    n_ft = 0
    index_map_shop[counter] = (shop_data.loc[i, "title"], shop_data.loc[i, "image_links"])
    vector_ft = np.zeros(100)
    for word in shop_data.loc[i, "text"]:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_shop.add_item(counter, vector_ft)
    counter += 1

ft_index_shop.build(10)
ft_index_shop.save('shop.ann')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


True

In [27]:
midf = np.mean(tfidf_vect.idf_)

In [21]:
 ft_index_shop = annoy.AnnoyIndex(100 ,'angular')
 ft_index_shop.load('shop.ann')

True

In [34]:
index_map_shop = {}
counter = 0
for i in tqdm_notebook(range(len(shop_data))):
    index_map_shop[counter] = (shop_data.loc[i, "title"], shop_data.loc[i, "image_links"])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [43]:
import pickle

with open("index_shop.pkl", "wb") as f:
    pickle.dump(index_map, f)

In [19]:
def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

In [22]:
ft_index_shop.get_nns_by_vector(np.zeros(100), 1, include_distances=True)

([176], [1.4142135381698608])

In [ ]:
updater = Updater(token='1428756614:AAGDiCk5A6VR1vzQ_zHY67maif_8XONS898') # Токен API к Telegram
dispatcher = updater.dispatcher

def startCommand(update: Update, context: CallbackContext):
    update.message.reply_text('Привет')

def textMessage(update: Update, context: CallbackContext):
    
    input_txt = preprocess_txt(update.message.text)
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = lr.predict(vect)
    
    if prediction[0] == 1:
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 5)
        for item in ft_index_shop_val:
            title, image = index_map_shop[item]
            update.send_message(chat_id=update.message.chat_id, text="title: {} image: {}".format(title, image))
        return
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
    if distances[0] > 0.2:
        print(distances[0])
        #update.send_message(chat_id=update.message.chat_id, text="Моя твоя не понимать")
        update.message.reply_text('Что?')
        return
    #update.send_message(chat_id=update.message.chat_id, text=index_map[ft_index_val[0]])
    update.message.reply_text(index_map[ft_index_val[0]])
        
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
updater.start_polling(clean=True)
updater.idle()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


0.305633008480072
0.6658725738525391


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


1.4142135381698608
1.4142135381698608
